In [58]:
import pandas as pd #Importing the PANDAS python library
import numpy as np #importing Numpy
%matplotlib inline 

#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #initiating VADER instance

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [59]:
def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)  #Calling the polarity analyzer
    print("{:-<40} {}".format(sentence, str(snt)))

In [88]:
Airbnb_listings_reviews = pd.read_csv('C:/Users/Yaarit/Documents/Insight/Insight_Project/Airbnb_reviews.csv',low_memory=False)

In [89]:
Airbnb_listings_reviews = Airbnb_listings_reviews[['listing_id', 'comments' ]] #this is all I need
Airbnb_listings_reviews.head()

,listing_id,comments
0,2539,Great host
1,2539,Nice room for the price. Great neighborhood. J...
2,2539,Very nice apt. New remodeled.
3,2539,Great place to stay for a while. John is a gre...
4,2539,.


In [90]:
len(Airbnb_listings_reviews)

1048575

In [91]:
Airbnb_listings_reviews['comments'].isnull().sum()

457

In [92]:
Airbnb_listings_reviews = Airbnb_listings_reviews.dropna()

In [102]:
len(Airbnb_listings_reviews)
len(Airbnb_listings_reviews.groupby(['listing_id']))

29247

In [124]:
words_of_kids = ['baby', 'toddler', 'babies', 'toddlers', 'infant', 'infants']
Airbnb_listings_reviews_parents = Airbnb_listings_reviews[Airbnb_listings_reviews.comments.str.contains('|'.join(words_of_kids ))]
#for word in words_of_kids:
  # Airbnb_listings_reviews['comments_kids'] = Airbnb_listings_reviews['comments'].str.find(word)

In [101]:
len(Airbnb_listings_reviews_parents.groupby(['listing_id']))
len(Airbnb_listings_reviews_parents)

4396

In [103]:
#%time   #to calulate the time it takes the algorithm to compute a VADER score
analyser = SentimentIntensityAnalyzer()

i=0 #counter

compval1 = [ ]  #empty list to hold our computed 'compound' VADER scores


while (i<len(Airbnb_listings_reviews_parents)):

    k = analyser.polarity_scores(Airbnb_listings_reviews_parents.iloc[i]['comments'])
    compval1.append(k['compound'])
    
    i = i+1
    
#converting sentiment values to numpy for easier usage

compval1 = np.array(compval1)

len(compval1)

4396

In [104]:
Airbnb_listings_reviews_parents['VADER score'] = compval1
Airbnb_listings_reviews_parents

C:\Users\Yaarit\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,listing_id,comments,VADER score
45,2595,Jennifer’s place is a wonderful retreat in the...,0.8514
210,3831,"This was my first time using an Airbnb, and I ...",0.9780
215,3831,Lisa’s apartment was perfect for our one night...,0.9810
265,3831,The 3rd floor is for guests only. You have to ...,0.5017
925,5203,"Gracious, warm and welcoming! My mother came t...",0.9808
1426,5803,We stayed at Laurie's place in Brooklyn for 6 ...,0.8718
1491,5803,Laurie's place was perfect for my stay - I was...,0.9622
1632,6021,"Our Stay at Claudio's was awesome,we had priva...",0.8276
1672,6021,I would first like to start by saying that Cla...,0.9973
1989,7097,"J's place was, as advertised, great for parent...",0.9790


In [56]:
analyser.polarity_scores(Airbnb_listings_reviews.iloc[1]['comments'])

{'neg': 0.0, 'neu': 0.601, 'pos': 0.399, 'compound': 0.9228}

In [123]:
Airbnb_listings_reviews_parents.iloc[15,1]

'Laurie\'s home was the perfect place for my nine days in Brooklyn. I was visiting my daughter/husband/and baby who live just a few blocks away. It was nice to come "home" to a quiet/comfy room after a long, busy day. It was also wonderful to have breakfast and coffee every morning. Right after I arrived she texted me to find out what I liked for breakfast. Plus, she has a wonderful espresso machine. Laurie was a great host. We only crossed paths a few times, which was fine, but she keeps in touch to make sure your stay is going well. Thank you Laurie!\r\n\r\nNov./Dec. 2012'